In [43]:
import re

num_lanes = 3
num_goals = 2
num_freecells = 2

In [49]:
F = lambda f,i: [f"F{i}",f"[F{f}]"][i==f]
L = lambda l,i: [f"L{i}",f"[L{l}|L{l}s]"][l==i]

F_ = lambda f,i: [f"F{i}","[]"][i==f]
L_ = lambda f,l,i: [f"L{i}",f"[F{f},L{l}|L{l}s]"][i==l]

f=2
l=3

res=f"""
f(G1, G2, {F(f,1)}, {F(f,2)}, {L(l,1)}, {L(l,2)}, {L(l,3)}, [S_|S], P, P_) :- (L{l} = b; (L{l} \= b, F{f} + 1 =:= L{l})), \+ member([{L_(f,l,1)},{L_(f,l,2)},{L_(f,l,3)}],P), step_str(F{f}, 'F{f}', 'L{l}', S_), f(G1, G2, {F_(f,1)}, {F_(f,2)}, {L_(f,l,1)}, {L_(f,l,2)}, {L_(f,l,3)}, S, [[{L_(f,l,1)},{L_(f,l,2)},{L_(f,l,3)}]|P], P_).
""".strip()

In [54]:
    def freecells_to_lanes(num_freecells, num_lanes):
        F = lambda f,i: [f"F{i}",f"[F{f}]"][i==f]
        L = lambda l,i: [f"L{i}",f"[L{l}|L{l}s]"][l==i]
        
        F_ = lambda f,i: [f"F{i}","[]"][i==f]
        L_ = lambda f,l,i: [f"L{i}",f"[F{f},L{l}|L{l}s]"][i==l]
        
        res = []
        for f in range(1,num_freecells+1):
            for l in range(1,num_lanes+1):
                res0 = f"""
                f(G1, G2,                                   % F{f} to L{l}
                    {F(f,1)}, {F(f,2)}, 
                    {L(l,1)}, {L(l,2)}, {L(l,3)}, 
                    [S_|S], P, P_
                ) :- 
                    (L{l} = b; (L{l} \= b, F{f} + 1 =:= L{l})), 
                    \+ member([{L_(f,l,1)},{L_(f,l,2)},{L_(f,l,3)}],P), 
                    step_str(F{f}, 'F{f}', 'L{l}', S_), 
                    f(G1, G2, 
                        {F_(f,1)}, {F_(f,2)}, 
                        {L_(f,l,1)}, {L_(f,l,2)}, {L_(f,l,3)}, 
                        S, [[{L_(f,l,1)},{L_(f,l,2)},{L_(f,l,3)}]|P], P_
                    ).
                """
                res.append(re.sub("\n            ","\n",res0).lstrip())
    
        return "\n".join(res)

In [56]:
print(freecells_to_lanes(num_freecells, num_lanes))

f(G1, G2, 
    [F1], F2, 
    [L1|L1s], L2, L3, 
    [S_|S], P, P_
) :- 
    (L1 = b; (L1 \= b, F1 + 1 =:= L1)), 
    \+ member([[F1,L1|L1s],L2,L3],P), 
    step_str(F1, 'F1', 'L1', S_), 
    f(G1, G2, 
        [], F2, 
        [F1,L1|L1s], L2, L3, 
        S, [[[F1,L1|L1s],L2,L3]|P], P_
    ).

f(G1, G2, 
    [F1], F2, 
    L1, [L2|L2s], L3, 
    [S_|S], P, P_
) :- 
    (L2 = b; (L2 \= b, F1 + 1 =:= L2)), 
    \+ member([L1,[F1,L2|L2s],L3],P), 
    step_str(F1, 'F1', 'L2', S_), 
    f(G1, G2, 
        [], F2, 
        L1, [F1,L2|L2s], L3, 
        S, [[L1,[F1,L2|L2s],L3]|P], P_
    ).

f(G1, G2, 
    [F1], F2, 
    L1, L2, [L3|L3s], 
    [S_|S], P, P_
) :- 
    (L3 = b; (L3 \= b, F1 + 1 =:= L3)), 
    \+ member([L1,L2,[F1,L3|L3s]],P), 
    step_str(F1, 'F1', 'L3', S_), 
    f(G1, G2, 
        [], F2, 
        L1, L2, [F1,L3|L3s], 
        S, [[L1,L2,[F1,L3|L3s]]|P], P_
    ).

f(G1, G2, 
    F1, [F2], 
    [L1|L1s], L2, L3, 
    [S_|S], P, P_
) :- 
    (L1 = b; (L1 \= b, F2 + 1 =:= L1